# Reproducing and trying to solve the bug in [this ticket](https://github.com/orgs/OpenMined/projects/81/views/1?pane=issue&itemId=28179093) - first notebook

In [1]:
import syft as sy
sy.requires(">=0.8,<0.8.1")

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/dk/Desktop/projects/PySyft


✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8 and the requirement <0.8.1


In [2]:
node = sy.orchestra.launch(name="test-domain-1", port=8080, 
                           dev_mode=True, reset=True)

Starting test-domain-1 server on 0.0.0.0:8080

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite



INFO:     Started server process [190963]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:48716 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:48716 - "POST /api/v1/new/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:48716 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:48718 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:48730 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:48716 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:48734 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:48716 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:48746 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:48752 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:48760 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:48768 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
I

In [3]:
domain_client = node.login(email="info@openmined.org", 
                           password="changethis")

#### Construct a `custom policy`

In [4]:
from typing import List, Dict, Any, Optional

class RepeatedCallPolicy(sy.CustomOutputPolicy):
    n_calls: int = 0
    downloadable_output_args: List[str] = []
    state: Dict[Any, Any] = {}

    def __init__(self, n_calls=1, downloadable_output_args: List[str] = None):
        self.downloadable_output_args = downloadable_output_args if downloadable_output_args is not None else []
        self.n_calls = n_calls + 1
        self.state = {"counts": 0}

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, context, outputs):
        output_dict = {}
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                output_dict[output_arg] = outputs[output_arg]

            self.state["counts"] += 1
        else:
            return None

        return output_dict

In [5]:
sy.CustomOutputPolicy??

In [6]:
policy = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])

In [7]:
print(f"{policy.n_calls = }")
print(f"{policy.downloadable_output_args = }")

policy.n_calls = 2
policy.downloadable_output_args = ['y']


In [8]:
policy.init_kwargs

{'n_calls': 1, 'downloadable_output_args': ['y']}

#### Make a pointer

In [9]:
import numpy as np
x = np.array([1,2,3])
x_pointer = sy.ActionObject.from_obj(x)
x_pointer

```python
Pointer
```
array([1, 2, 3])

In [10]:
domain_client.api.services.action.save(x_pointer)

<class 'syft.service.response.SyftSuccess'>: <class 'syft.service.action.numpy.NumpyArrayObject'> saved

In [11]:
x_pointer.id

<UID: 949e222917ec49c1a836854b5c5cc95b>

#### Make a function with the `RepeatedCallPolicy`

In [12]:
@sy.syft_function(
    input_policy=sy.ExactMatch(x=x_pointer),
    output_policy=RepeatedCallPolicy(n_calls=10, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x+1}

#### Request code execution

In [13]:
domain_client.api.services.code.request_code_execution(func)

```python
class Request:
  id: str = ad99e4c4f0f948fdb67a609fe2832d21
  requesting_user_verify_key: str = aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f
  approving_user_verify_key: str = None
  request_time: str = 2023-05-16 20:47:11
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "d0dd72cb7007a2b4cbd71b22563b0b78adce5759696da0443639dc2b2a54c332"
  changes: str = [syft.service.request.request.UserCodeStatusChange]

```

In [14]:
from syft import MessageStatus
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)
messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,5bc7391582da4750b1f859a3ca6bf108,Approval Request,MessageStatus.UNDELIVERED,2023-05-16 20:47:12,<<class 'syft.service.request.request.Request'...


In [15]:
from syft.service.request.request import ObjectMutation, UserCodeStatusChange, Request
func = None
request = None
for message in messages:
    req = message.linked_obj.resolve
    if isinstance(req, Request):
        for change in req.changes:
            if isinstance(change, UserCodeStatusChange):
                user_code = change.linked_obj.resolve
                if "func" in user_code.service_func_name:
                    func = user_code
                    request = req

In [16]:
request.changes

,type,id
0,syft.service.request.request.UserCodeStatusChange,ec0b004aa6c242e0a84bd570a55f7e58


In [17]:
print(f"{x_pointer = }")
result = func.unsafe_function(x=x_pointer)
result

x_pointer = Pointer:
[1 2 3]


{'y': array([2, 3, 4])}

In [18]:
final_result = request.accept_by_depositing_result(result) 
final_result

<class 'syft.service.response.SyftSuccess'>: Request ad99e4c4f0f948fdb67a609fe2832d21 changes applied

In [19]:
res = domain_client.api.services.code.func(x=x_pointer)
res

Pointer:
{'y': array([2, 3, 4])}

In [24]:
domain_client.api.services.request

,type,id,request_time,status,changes
0,syft.service.request.request.Request,6a3904ef94df42c0b1d4e2d4bb482530,2023-05-16 20:49:04,RequestStatus.PENDING,[syft.service.request.request.ActionStoreChange]
1,syft.service.request.request.Request,ad99e4c4f0f948fdb67a609fe2832d21,2023-05-16 20:47:11,RequestStatus.PENDING,[syft.service.request.request.UserCodeStatusCh...
2,syft.service.request.request.Request,f8ebe314e66a4bc882be54e9b2bbfb04,2023-05-16 20:47:13,RequestStatus.PENDING,"[syft.service.request.request.ObjectMutation, ..."


In [20]:
domain_client.api.services.messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,5bc7391582da4750b1f859a3ca6bf108,Approval Request,MessageStatus.UNDELIVERED,2023-05-16 20:47:12,<<class 'syft.service.request.request.Request'...
1,syft.service.message.messages.Message,2377d84165664e86a20ff238668c60f0,Approval Request,MessageStatus.UNDELIVERED,2023-05-16 20:47:13,<<class 'syft.service.request.request.Request'...


In [25]:
type(policy)

__main__.RepeatedCallPolicy

In [1]:
import syft as sy
from syft.service.policy.policy import CustomPolicy
from syft.service.policy.policy import CustomOutputPolicy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/dk/Desktop/projects/PySyft


In [2]:
byte = sy.serialize(CustomPolicy, to_bytes=True)
back = sy.deserialize(byte, from_bytes=True)
back == CustomPolicy

True

In [3]:
byte = sy.serialize(CustomOutputPolicy, to_bytes=True)
back = sy.deserialize(byte, from_bytes=True)
# back == CustomOutputPolicy

Exception: syft.service.policy.policy.CustomPolicy not in TYPE_BANK